In [69]:
!pip install pymongo

# 处理准备 数据

In [1]:
# environment
import pymongo
import neo4j
import tqdm

DATABASE = pymongo.MongoClient("192.168.1.222").temporary_token_similarity

Collection = DATABASE.token_similarity_v20240815


In [ ]:
# import sys
# import tqdm

# # 使用相对路径添加父目录到搜索路径
# sys.path.append("../gpc_demo") 
# sys.path.append("..") 

# from utils import calculate_cartesian_product_similarity, get_token, search_wikipedia, get_plaintext, calculate_similarity,query_partner_distancles


# from modelscope import AutoModel
# JinNaAI_Model = AutoModel.from_pretrained('jinaai/jina-embeddings-v2-base-zh', trust_remote_code=True) # trust_remote_code is needed to use the encode method
    
    

In [3]:
# function

# 按 google distance 获取样本数据
def query_gpc_limit_by_weight(weight_gt,weight_lte,fetch_size):
    driver = neo4j.GraphDatabase.driver(
            "bolt://192.168.1.229:17688",
            auth=("neo4j", "neo4j-test"),
        )
    with driver.session(database="neo4j") as session:
        result = session.run(
            "MATCH (start:P)-[r:D]->(end:P) "
            "WHERE r.weight > $weight_gt AND r.weight <= $weight_lte "
            "WITH start.Id as SID, start.Title as Stitle, end.Id as EID, end.Title as Etitle, r.weight as weight "
            "LIMIT $fetch_size "
            "RETURN SID, Stitle, EID, Etitle, weight"
            ,
            weight_gt=weight_gt,
            weight_lte=weight_lte,
            fetch_size=fetch_size
        )
        for record in result:
            yield record

mutiple=10000
delta = 1/mutiple
edge_list = []
node_list = []
for i in range(0,mutiple+1):
    gt = i/10000
    
    for doc in  query_gpc_limit_by_weight(gt,gt+delta,10):
        doc = dict(doc)
        doc['_id'] = f"{doc['SID']}-{doc['EID']}"
        edge_list.append(doc)
        node_list.append({
            '_id': doc['SID']
        })
try:
    collection_edge.insert_many(edge_list,ordered=False)
except pymongo.errors.BulkWriteError as bwe:
    print(bwe.details)
print(len(edge_list))
edge_list = []


99927


In [ ]:
# # 计算 title partner 的 token simmlarity, sts simmlarity {相似度如何}, 将结果update到 v1 key中 

from scipy.spatial.distance import (
    cosine,
    euclidean,
    minkowski,
    cityblock,
    chebyshev,
)

doc_list = []
for doc in Collection.find({'v1':None,'status':None}):
    doc_list.append(doc)
len(doc_list)



720059

In [ ]:
# SID, Stitle, EID, Etitle, weight
for doc in tqdm.tqdm(doc_list[25:]):

    plaintext_b = get_plaintext(doc['EID'])
    plaintext_a = get_plaintext(doc['SID'])
    
    if not plaintext_a or not plaintext_b:
        print('text miss')
        Collection.update_one({'_id': doc['_id']}, {'$set': {'status': 'text miss'}})
        continue
    
    if len(plaintext_a.split(" "))<=15 or len(plaintext_b.split(" "))<=15:
        print('text short', end=" ")
        # print(plaintext_a)
        # print(plaintext_b)
        Collection.update_one({'_id': doc['_id']}, {'$set': {'status': 'text short'}})
        continue
    
    token_set_1, _ = get_token(plaintext_a)
    token_set_2, _ = get_token(plaintext_b)
    
    df, _ = calculate_cartesian_product_similarity(
            tuple(token_set_1 - token_set_2), 
            tuple(token_set_2 - token_set_1),
            tuple(token_set_2 & token_set_1),
        )
    if not len(df):
        print('lost tokens')
        Collection.update_one({'_id': doc['_id']}, {'$set': {'status': 'lost tokens'}})
        continue
    
    token_similarity = df['similarity'].mean()
    embeddings = JinNaAI_Model.encode([plaintext_a, plaintext_b])
    cosine_similarity = cosine(embeddings[0],embeddings[1])
    euclidean_similarity = euclidean(embeddings[0],embeddings[1])
    minkowski_similarity = minkowski(embeddings[0],embeddings[1])
    cityblock_similarity = cityblock(embeddings[0],embeddings[1])
    chebyshev_similarity = chebyshev(embeddings[0],embeddings[1])
    
    Collection.update_one({'_id': doc['_id']},{"$set":{
        'v1':{
            'token':token_similarity.item(),
            'cosine': cosine_similarity.item(),
            'euclidean': euclidean_similarity,
            'minkowski': minkowski_similarity,
            'cityblock': cityblock_similarity.item(),
            'chebyshev': chebyshev_similarity.item(),
        }
    }})

# 从数据库中获取已经处理好的数据

In [ ]:
import pymongo

DATABASE = pymongo.MongoClient("192.168.1.222").temporary_token_similarity

Collection = DATABASE.token_similarity_v20240712

doc_list = []
for doc in Collection.find({'v1':{"$exists":True}}):
    doc_list.append({
        'source':doc['Stitle'],
        'dest':doc['Etitle'],
        'google similarity': 1-doc['weight'],
        'token similarity':doc['v1']['token'],
        'STS similarity':1-doc['v1']['cosine'],
    })
len(doc_list)


981425

In [ ]:

import pandas as pd
df_any = pd.DataFrame(doc_list)
df_any.to_csv("../model/score_1m.csv", index=False)
df_any
doc_list[0]

{'source': 'Counties_of_Iran',
 'dest': 'Provinces_of_Iran',
 'google similarity': 0.9995170648555507,
 'token similarity': 0.5066223437499999,
 'STS similarity': 0.610009006400506}

# subject 30K 导出

In [149]:
import pymongo
import pandas as pd

DATABASE = pymongo.MongoClient("192.168.1.222").temporary_token_similarity

Collection = DATABASE.token_similarity_v20240729

max_tree_struct = 0
doc_list = []
for doc in Collection.find({'v1':{"$exists":True}}):
    doc_list.append({
        'source':doc['STitle'],
        'dest':doc['ETitle'],
        'google similarity': 1-doc['weight'],
        'token similarity':doc['v1']['token'],
        'STS similarity':1-doc['v1']['cosine'],
        'Tree similarity':1-doc['v1']['tree_edit_distance']/255,
    })
    # max_tree_struct = max(max_tree_struct, doc['v1'][''])
len(doc_list)


31231

In [150]:
# doc

In [151]:


df_any = pd.DataFrame(doc_list)
df_any.to_csv("../model/subject_lv2.csv", index=False)
df_any
doc_list[0]

{'source': 'Geography',
 'dest': 'Linguistics',
 'google similarity': 0.5171631999999999,
 'token similarity': 0.4790892138095238,
 'STS similarity': 0.4055466218513175,
 'Tree similarity': 0.8901960784313725}

# subject 200 
处理 200 个 subject
放入数据库,进行分布计算

In [81]:
category_list = ['Computer engineering', 'Computer science', 'Psychology', 'Mathematics', 'Environmental engineering', 'Chemical engineering', 'Theoretical computer science', 'Sociology', 'Economics', 'Biological engineering', 'Deep learning', 'Anthropology', 'Medicine', 'Philosophy', 'Blockchains', 'Electrical engineering', 'Engineering disciplines', 'Genetic engineering', 'Materials science', 'Physics', 'Logic', 'Cognitive science', 'Mechanical engineering', 'Geography', 'Chemistry', 'Industrial engineering', 'Theoretical physics', 'Linguistics', 'Machine learning', 'Biology', 'Environmental science', 'Civil engineering', 'Artificial intelligence', 'Genome editing', 'Political science', 'Quantum computing', 'Neuroscience', 'Geology', 'Literature', 'History']
print(len(set(category_list)))

import neo4j

def query_title(title):
    driver = neo4j.GraphDatabase.driver(
            "bolt://192.168.1.227:17688",
            auth=("neo4j", "neo4j-test"),
        )
    record_list = []
    with driver.session(database="enwiki") as session:
        result = session.run(
            "MATCH (start:page {f_title: $source})-[r:page]->(end:page) "
            "RETURN start.pageId as SID, end.f_title as title, end.pageId as EID "
            ,
            source=title,
        )
        for record in result:
            record_list.append(record)

    return record_list

title_set = set()
for title in category_list:
    for record in query_title(title):
        title_set.add((title, int(record["SID"])))
        title_set.add((record["title"], int(record["EID"])))
len(title_set)

40


3968

In [80]:

import random
result_list = []

title_data_list = list(title_set)

tmp_list = []
for title_b, ID_b in title_data_list:
    if title_b in category_list:
        tmp_list.append((title_b, ID_b))
        
for title_b, ID_b in title_data_list:
    if title_b not in category_list:
        tmp_list.append((title_b, ID_b))
# tmp_list.extend(title_data_list[:162])
# random.shuffle(tmp_list)
print(len(tmp_list))

3968


In [78]:
# tmp_list

In [ ]:
!pip install psycopg

In [ ]:

import psycopg


def query_distance(list_set_1, list_set_2):
    pg_uri = os.environ.get('postgres_uri')
    ret = []
    with psycopg.connect(pg_uri) as conn:
        with conn.cursor() as cursor:
            for x in list_set_1:
                greater_than_or_equal_x = list(item for item in list_set_2 if item > x)
                if greater_than_or_equal_x:
                    cursor.execute("SELECT source_id,target_id,weight FROM edges WHERE source_id = %s AND target_id = ANY(%s);", 
                                (x, greater_than_or_equal_x,)
                                ) 
                    for result in cursor:
                        ret.append(result)
            for x in list_set_2:
                greater_than_or_equal_x = list(item for item in list_set_1 if item > x)
                if greater_than_or_equal_x:
                    cursor.execute("SELECT source_id,target_id,weight FROM edges WHERE source_id = %s AND target_id = ANY(%s);", 
                                (x, greater_than_or_equal_x,)
                                ) 
                    for result in cursor:
                        ret.append(result)
    return ret
                    
query_distance([123131,],[5043544])

[(123131, 5043544, 0.8404469)]

In [82]:
result_list = []
sz = len(tmp_list)
pbar = tqdm.tqdm()
for i in range(sz-1):
    for j in range(i+1, sz):
        # print(j)
        pbar.update(1)
        x = tmp_list[i]
        y = tmp_list[j]
        if x[1] == y[1]:
            continue
        if x[1] < y[1]:
            doc = {
                "SID": x[1],
                "EID": y[1],
                "STitle": x[0],
                "ETitle": y[0],
            }
        else:
            doc = {
                "SID": y[1],
                "EID": x[1],
                "STitle": y[0],
                "ETitle": x[0],
            }
        ret = query_distance([doc['SID'],],[doc['EID'],])
        if ret:
            doc['weight'] = ret[0][2]
            result_list.append(doc)
    if i > 40:
        break
    # break
len(result_list)

19900it [1:13:11,  4.53it/s]


31369

In [85]:

# environment
import pymongo
import tqdm

DATABASE = pymongo.MongoClient("192.168.1.222").temporary_token_similarity

Collection = DATABASE.token_similarity_v20240729

document_list = []
for doc in result_list:
    doc['_id'] = f"{doc['SID']}-{doc['EID']}"
    document_list.append(doc)
try:
    Collection.insert_many(document_list,ordered=False)
except pymongo.errors.BulkWriteError as bwe:
    print(bwe.details)



# test struct similarity

In [143]:
import requests

session = requests.Session()

url = 'http://192.168.1.227:10004/wiki_api/api/tree/editDistance'

# data = {
#     "stra":"[{'title':'a','edge':[[1,2],[1,33]]}]",
#     "strb":"[{'title':'b','edge':[[1,2],[1,3],[2,4]]}]",
#     "flag":0
# }
data = {'stra': '[{"title": "a", "edge": [[267487, 9127632], [267487, 14924067], [267487, 25879157], [216180, 267487], [216180, 9127632], [216180, 14924067], [216180, 25879157], [37436, 60931], [37436, 72132], [37436, 106238], [37436, 163390], [37436, 216180], [37436, 267487], [37436, 9127632], [37436, 14924067], [37436, 25879157], [21245, 22921], [21245, 26781], [21245, 37080], [21245, 37436], [21245, 60931], [21245, 72132], [21245, 106238], [21245, 163390], [21245, 216180], [21245, 267487], [21245, 9127632], [21245, 14924067], [21245, 25879157], [60931, 72132], [60931, 106238], [60931, 163390], [60931, 216180], [60931, 267487], [60931, 9127632], [60931, 14924067], [60931, 25879157], [106238, 163390], [106238, 216180], [106238, 267487], [106238, 9127632], [106238, 14924067], [106238, 25879157], [9127632, 14924067], [9127632, 25879157], [4805, 5664], [4805, 21245], [4805, 22921], [4805, 26781], [4805, 37080], [4805, 37436], [4805, 60931], [4805, 72132], [4805, 106238], [4805, 163390], [4805, 216180], [4805, 267487], [4805, 9127632], [4805, 14924067], [4805, 25879157], [22921, 26781], [22921, 37080], [22921, 37436], [22921, 60931], [22921, 72132], [22921, 106238], [22921, 163390], [22921, 216180], [22921, 267487], [22921, 9127632], [22921, 14924067], [22921, 25879157], [5664, 19447], [5664, 21245], [5664, 22921], [5664, 26781], [5664, 37080], [5664, 37436], [5664, 60931], [5664, 72132], [5664, 106238], [5664, 163390], [5664, 216180], [5664, 267487], [5664, 9127632], [5664, 14924067], [5664, 25879157], [72132, 106238], [72132, 163390], [72132, 216180], [72132, 267487], [72132, 9127632], [72132, 14924067], [72132, 25879157], [3717, 4805], [3717, 5664], [3717, 19447], [3717, 21245], [3717, 22921], [3717, 26781], [3717, 37080], [3717, 37436], [3717, 60931], [3717, 72132], [3717, 106238], [3717, 163390], [3717, 216180], [3717, 267487], [3717, 9127632], [3717, 14924067], [3717, 25879157], [163390, 216180], [163390, 267487], [163390, 9127632], [163390, 25879157], [26781, 37080], [26781, 37436], [26781, 60931], [26781, 72132], [26781, 106238], [26781, 163390], [26781, 216180], [26781, 267487], [26781, 9127632], [26781, 14924067], [26781, 25879157], [26781, 37080], [26781, 37436], [26781, 60931], [26781, 72132], [26781, 106238], [26781, 163390], [26781, 216180], [26781, 267487], [26781, 9127632], [26781, 14924067], [26781, 25879157], [14924067, 25879157], [216180, 267487], [216180, 9127632], [216180, 14924067], [216180, 25879157], [19447, 21245], [19447, 22921], [19447, 26781], [19447, 37080], [19447, 37436], [19447, 106238], [19447, 9127632], [19447, 14924067], [37080, 37436], [37080, 60931], [37080, 72132], [37080, 106238], [37080, 163390], [37080, 216180], [37080, 267487], [37080, 9127632], [37080, 14924067], [37080, 25879157], [267487, 9127632], [267487, 14924067], [267487, 25879157], [216180, 267487], [216180, 9127632], [216180, 14924067], [216180, 25879157], [37436, 60931], [37436, 72132], [37436, 106238], [37436, 163390], [37436, 216180], [37436, 267487], [37436, 9127632], [37436, 14924067], [37436, 25879157], [21245, 22921], [21245, 26781], [21245, 37080], [21245, 37436], [21245, 60931], [21245, 72132], [21245, 106238], [21245, 163390], [21245, 216180], [21245, 267487], [21245, 9127632], [21245, 14924067], [21245, 25879157], [60931, 72132], [60931, 106238], [60931, 163390], [60931, 216180], [60931, 267487], [60931, 9127632], [60931, 14924067], [60931, 25879157], [106238, 163390], [106238, 216180], [106238, 267487], [106238, 9127632], [106238, 14924067], [106238, 25879157], [9127632, 14924067], [9127632, 25879157], [4805, 5664], [4805, 21245], [4805, 22921], [4805, 26781], [4805, 37080], [4805, 37436], [4805, 60931], [4805, 72132], [4805, 106238], [4805, 163390], [4805, 216180], [4805, 267487], [4805, 9127632], [4805, 14924067], [4805, 25879157], [22921, 26781], [22921, 37080], [22921, 37436], [22921, 60931], [22921, 72132], [22921, 106238], [22921, 163390], [22921, 216180], [22921, 267487], [22921, 9127632], [22921, 14924067], [22921, 25879157], [5664, 19447], [5664, 21245], [5664, 22921], [5664, 26781], [5664, 37080], [5664, 37436], [5664, 60931], [5664, 72132], [5664, 106238], [5664, 163390], [5664, 216180], [5664, 267487], [5664, 9127632], [5664, 14924067], [5664, 25879157], [72132, 106238], [72132, 163390], [72132, 216180], [72132, 267487], [72132, 9127632], [72132, 14924067], [72132, 25879157], [3717, 4805], [3717, 5664], [3717, 19447], [3717, 21245], [3717, 22921], [3717, 26781], [3717, 37080], [3717, 37436], [3717, 60931], [3717, 72132], [3717, 106238], [3717, 163390], [3717, 216180], [3717, 267487], [3717, 9127632], [3717, 14924067], [3717, 25879157], [163390, 216180], [163390, 267487], [163390, 9127632], [163390, 25879157], [26781, 37080], [26781, 37436], [26781, 60931], [26781, 72132], [26781, 106238], [26781, 163390], [26781, 216180], [26781, 267487], [26781, 9127632], [26781, 14924067], [26781, 25879157], [26781, 37080], [26781, 37436], [26781, 60931], [26781, 72132], [26781, 106238], [26781, 163390], [26781, 216180], [26781, 267487], [26781, 9127632], [26781, 14924067], [26781, 25879157], [14924067, 25879157], [216180, 267487], [216180, 9127632], ]}]', 
        'strb': '[{"title": "b", "edge": [[295691, 670497], [295691, 711728], [295691, 3741945], [8221, 34341], [8221, 47643], [8221, 185285], [8221, 295691], [8221, 711728], [8221, 3741945], [47643, 295691], [47643, 646121], [47643, 670497], [47643, 711728], [47643, 3741945], [711728, 3741945], [6978, 8221], [6978, 34341], [6978, 47643], [6978, 295691], [6978, 670497], [6978, 711728], [6978, 3741945], [34341, 34549], [34341, 34578], [34341, 670497], [34578, 47643], [34549, 34578], [34549, 47643], [295691, 670497], [295691, 711728], [295691, 3741945], [8221, 34341], [8221, 47643], [8221, 185285], [8221, 295691], [8221, 711728], [8221, 3741945], [47643, 295691], [47643, 646121], [47643, 670497], [47643, 711728], [47643, 3741945], [711728, 3741945], [6978, 8221], [6978, 34341], [6978, 47643], [6978, 295691], [6978, 670497], [6978, 711728],]}]', 'flag': 0}

data = {'stra': '[{"title": "a", "edge": [[336271, 5912643], [336271, 208157], [5912643, 4063117], [18963754, 711288], [19005769, 2889], [20647050, 19555], [20647050, 20110874], [842224, 2396], [842224, 38393], [842224, 22122416], [6748280, 228108], [6748280, 208157], [22122416, 2889], [22122416, 711288], [605211, 2396], [711288, 228108], [208157, 9222], [2889, 15417], [5299, 13255], [5299, 27114], [5299, 19555], [6246, 19555], [6246, 15417], [6246, 19583], [19583, 23001]]}]', 'strb': '[{"title": "b", "edge": [[19653842, 26310], [19653842, 9236], [19653842, 13457], [19653842, 45086], [19653842, 9649], [71229276, 216186], [9236, 9127632], [9236, 646121], [9649, 18985062], [13457, 4250553], [38890, 9127632], [38890, 216186], [9127632, 17566205], [18985062, 4746766]]}]', 'flag': 0}

# data = {'stra': '[{"title": "a", "edge": [[267487, 9127632], [267487, 14924067], [267487, 25879157], ]}]', 
#         'strb': '[{"title": "b", "edge": [[295691, 670497], [295691, 711728], [295691, 3741945], ]}]', 
#         'flag': 0}

rep = session.post(url, json=data)
rep.json()


{'msg': 'success', 'data': {'a_b': 26}}

In [140]:
import networkx as nx


# 添加边来构建
# 添加边和权重

G = nx.Graph()
edges_with_weight = [
    (1, 2, 1), (1, 3, 3), (2, 4, 1), (21, 412, 10),
    (3, 4, 2), (4, 5, 6), (5, 6, 1)
]
G.add_weighted_edges_from(edges_with_weight)

# 找到所有连通分量
connected_components = list(nx.connected_components(G))

# 找到最大的连通分量
max_component = max(connected_components, key=len)

# 如果需要，可以创建一个新的图，只包含最大连通分量
max_subgraph = G.subgraph(max_component)

    
# 计算最小生成树
mst = nx.minimum_spanning_tree(max_subgraph)

# 打印最小生成树的边
print("最小生成树的边：")
for edge in mst.edges(data=False):
    print(edge)

最小生成树的边：
(1, 2)
(2, 4)
(3, 4)
(4, 5)
(5, 6)
